# Prepare points of interest (POI) data for bicycle networks analysis
## Project: Bicycle network analysis with Gourab, Sayat, Tyler, Michael, Roberta

This notebook downloads and prepares points of interest data to be used for bicycle network design, snapping them to the networks created in 01_prepare_networks:
* railway stations/halts and bus stops
* grid points
* ~~high population density points~~ (not yet implemented due to unreliable density data)

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-07-07  
Last modified: 2020-07-16

## Preliminaries

### Parameters

In [ ]:
debug = True # If True, will produce plots and/or verbose output to double-check

gridl = 1707 # in m, for generating the grid
# https://en.wikipedia.org/wiki/Right_triangle#Circumcircle_and_incircle
# 2*0.5 = a+a-sqrt(2)a   |   1 = a(2-sqrt2)   |   a = 1/(2-sqrt2) = 1.707
# This leads to a full 500m coverage when a (worst-case) square is being triangulated

snapthreshold = 500 # in m, tolerance for snapping POIs to network
bearingbins = 72 # number of bins to determine bearing. e.g. 72 will create 5 degrees bins

# We get all the railway stations/halts (including metro) and bus stops
poiparameters = {"railwaystation":{'railway':['station','halt']}#,
                 #"busstop":{'highway':'bus_stop'}
                }

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

In [ ]:
G_caralls = {}

## Download and wrangle data

### Railway stations/halts and bus stops

In [ ]:
for placeid, placeinfo in cities.items():
    print(placeid + ": Creating POIs")
    placepath = PATH["data"] + placeid + "/"
    if placeinfo["nominatimstring"] != '':
        location = ox.gdf_from_place(placeinfo["nominatimstring"])
        location = shapely.geometry.shape(location['geometry'][0])
    else:
        # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
        shp = fiona.open(placepath + placeid + ".shp")
        first = shp.next()
        location = shapely.geometry.shape(first['geometry'])
    
    cities[placeid]["bbox"] = location.bounds
    
    # We need the carall graph
    parameterdict = osmnxparameters['carall']
    G_carall = ox.graph_from_polygon(location, 
                               network_type = parameterdict['network_type'],
                               custom_filter = (parameterdict['custom_filter']),
                               retain_all = True,
                               simplify = False)
    
    for poiid, poitag in poiparameters.items():
        gdf = ox.pois.pois_from_polygon(location, tags = poitag)
        gdf = gdf[gdf['geometry'].type == "Point"] # only consider points, no polygons etc
        # Now snap to closest nodes in street network, save the nearest node ids
        nnids = set()
        for g in gdf['geometry']:
            n = ox.distance.get_nearest_node(G, [g.y, g.x])
            if not n in nnids and haversine((g.x, g.y), (G_carall.nodes[n]["x"], G_carall.nodes[n]["y"]))*1000 <= snapthreshold:
                nnids.add(n)
#         nnids = ox.distance.get_nearest_nodes(G_carall, gdf['geometry'].x, gdf['geometry'].y)
        with open(placepath + placeid + '_' + 'poi_' + poiid + '_nnidscarall.csv', 'w') as f:
            for item in nnids:
                f.write("%s\n" % item)
        
        gdf = gdf.apply(lambda c: c.astype(str) if c.name != 'geometry' else c, axis=0)
        try: # For some cities writing the gdf does not work (i.e. London, Manhattan)
            gdf.to_file(placepath + placeid + '_' + 'poi_' + poiid + '.gpkg', driver='GPKG')
        except:
            print("Notice: Writing the gdf did not work for " + placeid)
        if debug: gdf.plot(color='red')
            
    # Save to dict
    G_caralls[placeid] = G_carall

### Grid

Using code from: https://github.com/gboeing/osmnx-examples/blob/v0.11/notebooks/17-street-network-orientations.ipynb

In [ ]:
for placeid, placeinfo in cities.items():
    print(placeid + ": Creating grid")
    placepath = PATH["data"] + placeid + "/"
    
    G = G_caralls[placeid]
    
    
    # FIRST, determine the most common bearing, for the best grid orientation
    bearings = {}    
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))

    # weight bearings by length (meters)
    city_bearings = []
    for u, v, k, d in Gu.edges(keys = True, data = True):
        city_bearings.extend([d['bearing']] * int(d['length']))
    b = pd.Series(city_bearings)
    bearings = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

    bins = np.arange(bearingbins + 1) * 360 / bearingbins
    count = count_and_merge(bearingbins, bearings)
    principalbearing = bins[np.where(count == max(count))][0] 
    


    # SECOND, construct the grid

    # 1) Get lat lon window
    cities[placeid]["bbox"] = location.bounds

    # 2) Generate abstract grid points in window
    # https://stackoverflow.com/questions/40342355/how-can-i-generate-a-regular-geographic-grid-using-python
    # Set up projections
    p_ll = pyproj.Proj('+proj=longlat +datum=WGS84')
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    p_mt = pyproj.Proj(aeqd_proj.format(lat=location.centroid.y, lon=location.centroid.x)) # equidistant projection

    # Make the area larger to account for tilted grid
    deltax = cities[placeid]["bbox"][2] - cities[placeid]["bbox"][0]
    deltay = cities[placeid]["bbox"][3] - cities[placeid]["bbox"][1]

    # Create corners of rectangle to be transformed to a grid
    sw = shapely.geometry.Point((cities[placeid]["bbox"][0], cities[placeid]["bbox"][1]))
    ne = shapely.geometry.Point((cities[placeid]["bbox"][2]+10*deltax, cities[placeid]["bbox"][3]+10*deltay))
    
    # Project corners to target projection
    transformed_sw = pyproj.transform(p_ll, p_mt, sw.x, sw.y) # Transform NW point to equidistant
    transformed_ne = pyproj.transform(p_ll, p_mt, ne.x, ne.y) # .. same for SE

    # Iterate over 2D area
    principalbearing = principalbearing % 90 # Take it modulo 90 because it will be a square grid
    if principalbearing > 45:
        principalbearing -= 90 # Make bearing fall btw -45 and 45

    xcoords = np.arange(transformed_sw[0], transformed_ne[0], gridl)
    ycoords = np.arange(transformed_sw[1], transformed_ne[1], gridl)
    xsize =  xcoords.size
    ysize = ycoords.size
    xcoords = np.tile(xcoords, ysize)
    ycoords = np.repeat(ycoords, xsize)
    gridpoints=[(x, y) for x, y in zip(xcoords, ycoords)]
    new_points = rotate_grid(gridpoints, origin = transformed_sw, degrees = principalbearing)
    
    # https://stackoverflow.com/questions/42459068/projecting-a-numpy-array-of-coordinates-using-pyproj
    fx, fy = pyproj.transform(p_mt, p_ll, new_points[:,0], new_points[:,1])
    gridpoints = np.dstack([fx, fy])[0]
    if principalbearing >=0:
        # If we rotated right, we need to shift everything to the left
        gridpoints[:,0] -= 4*deltax*math.sin(np.deg2rad(principalbearing))
    else:
        # If we rotated left, we need to shift everything down and to the right
        gridpoints[:,0] += 4*deltax*math.sin(np.deg2rad(principalbearing))
        gridpoints[:,1] -= 4*deltay

    # Cut back to bounding box
    mask = (gridpoints[:,0] >= cities[placeid]["bbox"][0]) & (gridpoints[:,0] <= cities[placeid]["bbox"][2]) & (gridpoints[:,1] >= cities[placeid]["bbox"][1]) & (gridpoints[:,1] <= cities[placeid]["bbox"][3])
    gridpoints_cut = gridpoints[mask]
    
    if debug:
        fig = plt.figure(figsize=[2*6.4, 2*4.8])
        ax = fig.add_axes([0, 0, 1, 1])
        ax.set_aspect('equal', adjustable = 'datalim')
        plt.plot([g[0] for g in gridpoints_cut], [g[1] for g in gridpoints_cut], ".", color = "red")

    # 3) Snap grid points to map
    nnids = set()
    for g in gridpoints_cut:
        n = ox.distance.get_nearest_node(G, [g[1], g[0]])
        if not n in nnids and haversine((g[0], g[1]), (G.nodes[n]["x"], G.nodes[n]["y"]))*1000 <= snapthreshold:
            nnids.add(n)
    with open(placepath + placeid + '_poi_grid_nnidscarall.csv', 'w') as f:
        for item in nnids:
            f.write("%s\n" % item)

### Population density (for now commented out as unfinished: data not reliable enough)

In [ ]:
# # https://medium.com/@danielmoyo/raster-data-in-python-part-i-reading-and-visualising-raster-data-95cb072fcbfa
# import rasterio

# densityfolder = "populationdensity"
# for placeid, placeinfo in cities.items():
#     raster_path = PATH["data"] + densityfolder + "/" + placeinfo['countryid'] + "_ppp_2020_UNadj.tif"
#     pop_raster = rasterio.open(raster_path)
    
#     cities[placeid]["bbox"] = location.bounds

#     def plot_raster(arr, title=None, figsize=(30,10)):
#         """ helper function to plot a raster array """
#         fig, ax = plt.subplots(figsize=figsize)
#         plt.imshow(arr)
#         plt.colorbar()
#         plt.title(f'{title}')
#         plt.xlabel('Column #')
#         plt.ylabel('Row #')
#         return fig, ax

#     pop_data = pop_raster.read(1)
#     pop_data[pop_data<0] = 0
#     pop_count = pop_data[pop_data>0].sum()
#     plot_raster(pop_data, title=pop_count)

In [ ]:
# # https://stackoverflow.com/questions/50191648/gis-geotiff-gdal-python-how-to-get-coordinates-from-pixel
# # https://gdal.org/api/gdaldataset_cpp.html#_CPPv4N11GDALDataset15GetGeoTransformEPd

# # Problems:
# # https://gis.stackexchange.com/questions/331908/notimplementederror-wrong-number-or-type-of-arguments-for-overloaded-function
# # https://gis.stackexchange.com/questions/44040/osr-python-testsr-importfromepsg4326-returns-empty-spatial-reference
# # https://www.programcreek.com/python/example/58591/osgeo.osr.SpatialReference


# def generate_latlons(raster_path):
#     # open the dataset and get the geo transform matrix
#     ds = gdal.Open(raster_path) 
#     xoffset, px_w, rot1, yoffset, px_h, rot2 = ds.GetGeoTransform()
#     ds_arr = ds.ReadAsArray()
    
#     x_arr = np.arange(0, ds_arr.shape[0], 1)
#     x_arr.shape = (len(x_arr), 1)
#     x_arr = np.repeat(x_arr, [ds_arr.shape[1]], axis=1)
    
#     y_arr = np.arange(0, ds_arr.shape[1], 1)
#     y_arr.shape = (1, len(y_arr))
#     y_arr = np.repeat(y_arr, [ds_arr.shape[0]], axis=0)
    
#     # supposing x and y are your pixel coordinate this 
#     # is how to get the coordinate in space.
#     lon_arr = xoffset + px_w * y_arr + rot1 * x_arr + px_w / 2.0
#     lat_arr = yoffset + px_h * y_arr + rot2 * x_arr + px_h / 2.0
    
#     return(lat_arr, lon_arr)

# for placeid, placeinfo in cities.items():
#     raster_path = PATH["data"] + densityfolder + "/" + placeinfo['countryid'] + "_ppp_2020_UNadj.tif"
#     pop_raster = rasterio.open(raster_path)
#     (lat, lon) = generate_latlons(raster_path)
    
# # To do: only read the relevant window
# # https://medium.com/@danielmoyo/raster-data-in-python-part-ii-working-with-geographic-coordinates-a368708de5f2

In [ ]:
# indices = np.where(pop_data == pop_data.max())
# c_x = indices[0].item()
# c_y = indices[1].item()

# delta = 60
# plot_raster(pop_data[c_x-delta:c_x+delta, c_y-delta:c_y+delta], title="Population density, Copenhagen")